<a href="https://colab.research.google.com/github/renanpyd/Course_Google_Colab/blob/main/site/pt-br/tutorials/load_data/pandas_dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Carregar um pandas.DataFrame

Este tutorial fornece um exemplo de como carregar dataframe do pandas em um `tf.data.Dataset`.

Este tutorial usa um pequeno [conjunto de dados] (https://archive.ics.uci.edu/ml/datasets/heart+Disease) fornecido pela Cleveland Clinic Foundation for Heart Disease. Existem várias centenas de linhas no CSV. Cada linha descreve um paciente e cada coluna descreve um atributo. Usaremos essas informações para prever se um paciente tem uma doença cardíaca, que neste conjunto de dados é uma tarefa de classificação binária.

## Ler os dados usando pandas

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import pandas as pd
import tensorflow as tf

Fazer download do arquivo csv que contém o conjunto de dados do coração.

In [3]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/applied-dl/heart.csv')

Exception: ignored

Ler o arquivo csv usando pandas.

In [ ]:
df = pd.read_csv(csv_file)

In [ ]:
df.head()

In [ ]:
df.dtypes

Converta a coluna `thal`, que é um `objeto` no dataframe para um valor numérico discreto

In [ ]:
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df.thal.cat.codes

In [ ]:
df.head()

## Carregar dados usando o `tf.data.Dataset`

Use `tf.data.Dataset.from_tensor_slices` para ler os valores de um dataframe do pandas.

Uma das vantagens do uso do `tf.data.Dataset` é que ele permite escrever pipelines de dados simples e altamente eficientes. Leia o [loading data guide] (https://www.tensorflow.org/guide/data) para obter mais informações.

In [ ]:
target = df.pop('target')

In [4]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

NameError: ignored

In [ ]:
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Como um `pd.Series` implementa o protocolo `__array__`, ele pode ser usado de forma transparente em praticamente qualquer lugar que você usaria um `np.array` ou um `tf.Tensor`.

In [ ]:
tf.constant(df['thal'])

Aleatório e lote do conjunto de dados.

In [ ]:
train_dataset = dataset.shuffle(len(df)).batch(1)

## Crirar e treinar um modelo

In [ ]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [ ]:
model = get_compiled_model()
model.fit(train_dataset, epochs=15)

## Alternativa para colunas de características

Passar um dicionário como entrada para um modelo é tão fácil quanto criar um dicionário correspondente de camadas `tf.keras.layers.Input`, aplicar qualquer pré-processamento e empilhá-los usando a [API funcional] (../../guide/keras/functional.ipynb). Você pode usar isso como uma alternativa para [colunas de características] (../keras/feature_columns.ipynb).

In [ ]:
inputs = {key: tf.keras.layers.Input(shape=(), name=key) for key in df.keys()}
x = tf.stack(list(inputs.values()), axis=-1)

x = tf.keras.layers.Dense(10, activation='relu')(x)
output = tf.keras.layers.Dense(1)(x)

model_func = tf.keras.Model(inputs=inputs, outputs=output)

model_func.compile(optimizer='adam',
                   loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=['accuracy'])

A maneira mais fácil de preservar a estrutura da coluna de um `pd.DataFrame` quando usado com `tf.data` é converter o `pd.DataFrame` em um `dict` e dividir esse dicionário.

In [ ]:
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values)).batch(16)

In [ ]:
for dict_slice in dict_slices.take(1):
  print (dict_slice)

In [ ]:
model_func.fit(dict_slices, epochs=15)